In [ ]:
import pandas as pd
import sys, os

module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from crimebb import *

In [ ]:
YEAR="2021"

In [ ]:
DATA_PATH="../../data/"
CSV_PROCESSED = f"{DATA_PATH}csv/{YEAR}/processed/"

### Exploring data

#### members

In [ ]:
members_df = pd.read_csv(f"{CSV_PROCESSED}members.csv", sep="\t", low_memory=False)
members_df

#### websites

In [ ]:
website_df = pd.read_csv(f"{CSV_PROCESSED}sites.csv", sep="\t", low_memory=False)
website_df

#### boards

In [ ]:
boards_df = pd.read_csv(f"{CSV_PROCESSED}boards.csv", sep="\t", low_memory=False)
boards_df

#### threads

In [ ]:
threads_df = pd.read_csv(f"{CSV_PROCESSED}threads.csv", sep="\t", low_memory=False)
threads_df

#### Posts

In [ ]:
chunk_size = 1000000

posts_reader = pd.read_csv(f"{CSV_PROCESSED}posts.csv", sep="\t", low_memory=False, iterator=True)
            
posts_df = pd.DataFrame()

len_readed=chunk_size
while len_readed>=chunk_size:
    cur_posts_df = posts_reader.get_chunk(chunk_size).copy()
    cur_posts_df.drop_duplicates(inplace=True)

    posts_df = pd.concat([posts_df, cur_posts_df], ignore_index=True)

    len_readed = cur_posts_df.shape[0]

In [ ]:
posts_df.info()

### Removing none values

#### Members

In [ ]:
members_df = members_df[ (members_df["username"].str.lower()!="none")].copy().drop_duplicates()
members_df = members_df[ (~members_df["username"].isna()) ].copy().drop_duplicates()

members_df

#### Threads

In [ ]:
threads_df = threads_df[ (threads_df["username"].str.lower()!="none") ].copy().drop_duplicates()
threads_df = threads_df[ (~threads_df["username"].isna()) ].copy().drop_duplicates()

threads_df

#### Posts

In [ ]:
posts_df = posts_df[ (posts_df["username"].str.lower()!="none") ].copy().drop_duplicates()
posts_df = posts_df[ (~posts_df["username"].isna()) ].copy().drop_duplicates()

posts_df

### Members x Sites

In [ ]:
member_website = pd.merge(members_df[["username", "site_id"]].drop_duplicates(),
                          boards_df[["site_id", "site_name"]].drop_duplicates(),
                          on="site_id", how="left").drop_duplicates()
member_website

### Site x Member

In [ ]:
site_users_df = pd.pivot_table(member_website,
                 index=["site_name"],
                 values=["username"],
                 aggfunc={
                     "username":len
                 }
                ).sort_values(by="username", ascending=False).reset_index(level=0)
site_users_df.set_index('site_name', inplace=True)
site_users_df.rename(columns={"username":"num_users"}, inplace=True)
site_users_df

In [ ]:
mean_val = round(site_users_df["num_users"].mean(), 2)

ax = site_users_df.plot(kind="bar", figsize=(16,8), title=f"Número de usuarios por website, Avg: {mean_val}", xlabel="Site name", ylabel="usuarios", rot=45)
# ax.legend(list(temp_df["site_name"]))
ax.get_legend().remove()

### Boards

topic x website

In [ ]:
topics_website_df = pd.pivot_table(boards_df[["site_name", "board_title"]].drop_duplicates(),
                                 index=["site_name"],
                                 values=["board_title"],
                                 aggfunc={
                                     "board_title":len
                                 }
                                ).sort_values(by="board_title", ascending=False).reset_index(level=0)
topics_website_df.set_index('site_name', inplace=True)
topics_website_df.rename(columns={"board_title":"num_boards_created"}, inplace=True)
topics_website_df

In [ ]:
mean_val = round(topics_website_df["num_boards_created"].mean(), 2)

ax = topics_website_df.plot(kind="bar", figsize=(16,8), title=f"Número de topicos por website, Avg: {mean_val}", xlabel="Site name", ylabel="usuarios", rot=45)
# ax.legend(list(temp_df["site_name"]))
ax.get_legend().remove()

### Boards x Members

##### usermane x website

In [ ]:
users_sites_df = pd.pivot_table(member_website,
                 index=["username"],
                 values=["site_name"],
                 aggfunc={
                     "site_name":len
                 }
                ).sort_values(by="site_name", ascending=False).reset_index(level=0)
users_sites_df.rename(columns={"site_name":"num_websites_registered"}, inplace=True)
users_sites_df.set_index("username", inplace=True)
users_sites_df

In [ ]:
users_sites_ = users_sites_df.pivot_table(columns=['num_websites_cadastrado'], aggfunc='size')
users_sites_

In [ ]:
ax = users_sites_.plot(kind="bar", figsize=(16,8), title=f"Frequencia do número de páginas interagidas por usuarios, Avg: {users_sites_.mean()}", xlabel="Freqeuencia de usuarios cadastrados por pagina", ylabel="Quantidade de usuarios", rot=0)
# ax.legend(list(temp_df["site_name"]))
#ax.get_legend().remove()

### Threads

#### Threads x Boards

In [ ]:
board_threads_df = pd.merge(threads_df, boards_df[["board_id", "site_id", "site_name", "board_title"]], how="left", on=["site_id", "board_id"])
board_threads_df

In [ ]:
site_threads_df = pd.pivot_table(board_threads_df,
                 index=["site_name"],
                 values=["thread_id"],
                 aggfunc={
                     "thread_id":len
                 }
                ).sort_values(by="thread_id", ascending=False).reset_index(level=0)
site_threads_df.set_index('site_name', inplace=True)
site_threads_df.rename(columns={"thread_id":"num_threads_created"}, inplace=True)
site_threads_df

In [ ]:
mean_val = round(site_threads_df["num_threads_created"].mean(), 2)

ax = site_threads_df.plot(kind="bar", figsize=(16,8), title=f"Número de threads por website, Avg: {mean_val}", xlabel="Site name", ylabel="Num threads", rot=45)
# ax.legend(list(temp_df["site_name"]))
ax.get_legend().remove()

### Threads x Members

o usuário 1 criou muitos threads, em geral todos os usuários criaram 1 thread. 

In [ ]:
thread_member_df = pd.pivot_table(board_threads_df[["site_name", "thread_id", "username"]].drop_duplicates(),
                                 index=["username", "site_name"],
                                 values=["thread_id"],
                                 aggfunc={
                                     "thread_id":len
                                 }
                                ).sort_values(by="thread_id", ascending=False)#.reset_index(level=0)
thread_member_df.rename(columns={"thread_id":"num_threads_created"}, inplace=True)
thread_member_df

In [ ]:
thread_member_ = thread_member_df.pivot_table(columns=['num_threads_created'], aggfunc='size')
thread_member_.sort_values(ascending=False, inplace=True)
thread_member_

In [ ]:
mean_val = round(thread_member_.mean(), 2)

ax = thread_member_[:100].plot(kind="bar", figsize=(16,8), title=f"Frequencia dos numeros de threads criadas por usuario/website, Avg: {mean_val}", xlabel="Número de threads criados", ylabel="Frequencia de threads", rot=90)
# ax.legend(list(temp_df["site_name"]))
#ax.get_legend().remove()

### Threads x Boards

In [ ]:
thread_board_df = pd.pivot_table(board_threads_df[["site_name", "thread_id", "board_title", "board_id"]].drop_duplicates(),
                              index=["site_name", "board_title", "board_id"],
                              values=["thread_id"],
                              aggfunc={
                                 "thread_id":len
                              }).sort_values(by="thread_id", ascending=False)
thread_board_df.rename(columns={"thread_id":"num_threads_created"}, inplace=True)
thread_board_df.reset_index(inplace=True)
thread_board_df.set_index(["site_name", "board_title"], inplace=True)
thread_board_df.drop(columns=['board_id'], inplace=True)
thread_board_df

In [ ]:
thread_board_df_ = thread_board_df.pivot_table(columns=['num_threads_created'], aggfunc='size')
thread_board_df_.sort_values(ascending=False, inplace=True)
thread_board_df_

In [ ]:
mean_val = round(thread_board_df_.mean(), 2)

ax = thread_board_df_[:100].plot(kind="bar", figsize=(16,8), title=f"Número de threads criadas por tematica/website, Avg: {mean_val}", xlabel="Frequencia dos numeros de threads criados", ylabel="Frequencia de threads", rot=90)
# ax.legend(list(temp_df["site_name"]))
#ax.get_legend().remove()

### Posts

#### Posts x Website

In [ ]:
website_df

In [ ]:
posts_website_df = pd.merge(posts_df, website_df, how="left", on="site_id")
posts_website_df

In [ ]:
post_x_website = pd.pivot_table(posts_website_df[["site_name", "post_id"]].drop_duplicates(),
                              index=["site_name"],
                              values=["post_id"],
                              aggfunc={
                                 "post_id":len
                              }).sort_values(by="post_id", ascending=False)
post_x_website.rename(columns={"post_id":"num_posts_created"}, inplace=True)
post_x_website.reset_index(inplace=True)
post_x_website.set_index(["site_name"], inplace=True)
post_x_website

In [ ]:
mean_val = round(post_x_website["num_posts_created"].mean(), 2)

ax = post_x_website.plot(kind="bar", figsize=(16,8), title=f"Número de posts por website, Avg: {mean_val}", xlabel="Site name", ylabel="Num posts", rot=45)
# ax.legend(list(temp_df["site_name"]))
ax.get_legend().remove()

#### Posts x members

In [ ]:
post_member_df = pd.pivot_table(posts_website_df[["site_name", "post_id", "username"]].drop_duplicates(),
                                 index=["username", "site_name"],
                                 values=["post_id"],
                                 aggfunc={
                                     "post_id":len
                                 }
                                ).sort_values(by="post_id", ascending=False)#.reset_index(level=0)
post_member_df.rename(columns={"post_id":"num_posts_created"}, inplace=True)
post_member_df

In [ ]:
post_member_ = post_member_df.pivot_table(columns=['num_posts_created'], aggfunc='size')
post_member_.sort_values(ascending=False, inplace=True)
post_member_

In [ ]:
mean_val = round(post_member_.mean(), 2)

ax = post_member_[:100].plot(kind="bar", figsize=(16,8), title=f"Frequencia dos numeros de posts criados por usuario/website, Avg: {mean_val}", xlabel="Número de posts criados", ylabel="Frequencia de posts", rot=90)
# ax.legend(list(temp_df["site_name"]))
#ax.get_legend().remove()

#### Posts x Boards

In [ ]:
posts_boards_df = pd.merge(posts_website_df, boards_df[["site_id", "board_id", "board_title"]].drop_duplicates(), on=["site_id", "board_id"], how="left")
posts_boards_df

In [ ]:
post_board_df = pd.pivot_table(posts_boards_df[["site_name", "board_title", "board_id", "post_id"]].drop_duplicates(),
                                 index=["site_name", "board_title", "board_id"],
                                 values=["post_id"],
                                 aggfunc={
                                     "post_id":len
                                 }
                                ).sort_values(by="post_id", ascending=False)#.reset_index(level=0)
post_board_df.rename(columns={"post_id":"num_posts_created"}, inplace=True)
post_board_df.reset_index(inplace=True)
post_board_df.set_index(["site_name", "board_title"], inplace=True)
post_board_df.drop(columns=['board_id'], inplace=True)
post_board_df

In [ ]:
post_board_df_ = post_board_df.pivot_table(columns=['num_posts_created'], aggfunc='size')
post_board_df_.sort_values(ascending=False, inplace=True)
post_board_df_

In [ ]:
mean_val = round(post_board_df_.mean(), 2)

ax = post_board_df_[:100].plot(kind="bar", figsize=(16,8), title=f"Número de posts criadas por tematica/website, Avg: {mean_val}", xlabel="Frequencia dos numeros de posts criados", ylabel="Frequencia de posts", rot=90)
# ax.legend(list(temp_df["site_name"]))
#ax.get_legend().remove()

#### Posts x Threads

In [ ]:
posts_threads_df = pd.merge(posts_boards_df, threads_df[["site_id", "board_id", "thread_id", "thread_title"]].drop_duplicates(), on=["site_id", "board_id", "thread_id"], how="left")
posts_threads_df

In [ ]:
post_thread_board_df = pd.pivot_table(posts_threads_df[["site_name", "thread_title", "thread_id", "post_id"]].drop_duplicates(),
                                 index=["site_name", "thread_title", "thread_id"],
                                 values=["post_id"],
                                 aggfunc={
                                     "post_id":len
                                 }
                                ).sort_values(by="post_id", ascending=False)#.reset_index(level=0)
post_thread_board_df.rename(columns={"post_id":"num_posts_created"}, inplace=True)
post_thread_board_df.reset_index(inplace=True)
post_thread_board_df.set_index(["site_name", "thread_title"], inplace=True)
post_thread_board_df.drop_duplicates(inplace=True)
post_thread_board_df.drop(columns=['thread_id'], inplace=True)
post_thread_board_df

In [ ]:
post_threads_board_df_ = post_thread_board_df.pivot_table(columns=['num_posts_created'], aggfunc='size')
post_threads_board_df_.sort_values(ascending=False, inplace=True)
post_threads_board_df_

In [ ]:
mean_val = round(post_threads_board_df_.mean(), 2)

ax = post_threads_board_df_[:100].plot(kind="bar", figsize=(16,8), title=f"Número de posts criadas por thread/website, Avg: {mean_val}", xlabel="Frequencia dos numeros de posts criados", ylabel="Frequencia de posts", rot=90)
# ax.legend(list(temp_df["site_name"]))
#ax.get_legend().remove()